# Sentiment Analysis
***
## Table of Contents

***

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import string
import time
import re
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy
from typing import Dict, List, Optional
from numpy.typing import NDArray

## 1. Introduction

## 2. Device Agnostic Code
Mac GPU acceleration (`mps` backend) delivers significant speed-up over CPU for deep learning tasks, especially for large models and batch sizes. On Windows, `cuda` is used instead of `mps`.

`NotImplementedError: The operator 'aten::_nested_tensor_from_mask_left_aligned' is not currently implemented for the MPS device.`

As of current PyTorch version, certain Transformer operations (especially those involving masks in nn.TransformerEncoder) are not supported natively on Apple Silicon (MPS).

In [ ]:
# device = torch.device(
#     "mps"  # MacOS
#     if torch.backends.mps.is_available()
#     else "cuda"  # Windows
#     if torch.cuda.is_available()
#     else "cpu"  # No GPU Available
# )
device = torch.device("cpu")

## 3. Loading Data

The dataset used in this project (retrieved from [Kaggle - IMDB Dataset of 50K Movie Reviews](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)) includes:

- **review**: Review comments in text.
- **sentiment**: Whether the review is positive or negative.

In [ ]:
df = pd.read_csv("_datasets/IMDB_Dataset.csv")

In [ ]:
df.head()

In [ ]:
print("=" * 50)
print(f"Shape of the dataset: {df.shape}")
print("=" * 50)
print(f"Count of null values: {df.isnull().sum().sum()}")

In [ ]:
df.info()

## 3. Data Preprocessing
1. Text Cleaning
    - Lower all letters
    - Removing HTML Tags
    - Removing URLs
    - Removing Emojis and Non-ASCII Characters
    - Remove Punctuations
    - Remove extra whitespace
2. Tokenisation
3. Building Vocabulary and Mapping Tokens to Indices

### Text Cleaning

In [ ]:
string.punctuation

In [ ]:
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"http\S+|www\.\S+", "", text)
    text = re.sub(r"[^\x00-\x7F]+", "", text)
    text = re.sub("[{}]".format(re.escape(string.punctuation)), "", text)
    text = re.sub(r"\s+", " ", text).strip()  # Leave a space between words
    return text

In [ ]:
df["clean_text"] = df["review"].apply(clean_text)
df.head()

In [ ]:
df["n_tokens"] = df["clean_text"].apply(lambda text: len(text.split()))

In [ ]:
df.head()

### Tokenisation
Split all reviews into tokens (words). 

In [ ]:
all_words = [token for text in df["clean_text"] for token in text.split()]

In [ ]:
print(all_words[:10])

### Building Vocabulary and Mapping Tokens to Indices
Using `Counter()` allows us to get the frequency of each word, sort in descending order (we can specify `n` parameter to extract the top N most frequent words).
Then we assign a unique index to each word, create mapping (word2index), and reserve indices for padding (`<PAD>`) and unknown (`<UNK>`) tokens.

**Padding**:
- Padding is the process of adding special tokens (usually represented as `<PAD>`) to sequences so that all sequences in a batch have the same length.
- This is necessary because neural networks, especially in libraries like PyTorch, require inputs to be in tensors of consistent shape.

*Example*:
- Original:
    - ["i", "loved", "this", "movie"]
- After padding to length 6:
    - ["i", "loved", "this", "movie", "`<PAD>`", "`<PAD>`"]

**Unknown**:
- `<UNK>` stands for '*unknown token*', serving as a placeholder for any token (word) in the input text that does not exist in the vocabulary.

*Example*:
- Vocabulary:
    - { "the":2, "movie":3, "`<PAD>`":0, "`<UNK>`":1 }
- Input:
    - "the plot was amazing" -> ["the", "`<UNK>`", "`<UNK>`", "`<UNK>`"]

In [ ]:
word_counts = Counter(all_words)

In [ ]:
all_words_sorted = word_counts.most_common()

In [ ]:
word2index = {word: i for i, (word, counts) in enumerate(all_words_sorted, start=2)}
word2index["<PAD>"] = 0
word2index["<UNK>"] = 1

In [ ]:
print(f"Number of unique words: {len(word2index) - 2}")

In [ ]:
# Convert text to int sequences
def text_to_int(text: str, word2index: Dict[str, int]) -> List[int]:
    """
    Convert a text string into a list of integer indices based on the word2index mapping.

    Args:
        text: The input text string.
        word2index: Dictionary mapping words to integer indices.

    Returns:
        List of integer indices corresponding to words in the input text.
        Unknown words are mapped to the index of the "<UNK>" token.
    """
    return [word2index.get(word, word2index["<UNK>"]) for word in text.split()]


def pad_or_truncate(text: List[int], max_len: int, pad_value: int = 0) -> List[int]:
    """
    Pad or truncate a list of integer indices to a specified maximum length.

    Args:
        text: List of word indices representing the text.
        max_len: The desired length of the output list.
        pad_value: The value to use for padding. Defaults to 0.

    Returns:
        The input list truncated or padded to max_len.
    """
    if len(text) >= max_len:  # Text length ok
        return text[:max_len]
    else:  # Text too short -> Add padding
        return text + [pad_value] * (max_len - len(text))

In [ ]:
SEQUENCE_LENGTH = 128

all_review_seq = [
    pad_or_truncate(text_to_int(text, word2index), SEQUENCE_LENGTH, word2index["<PAD>"])
    for text in df["clean_text"]
]

In [ ]:
print(all_review_seq[9])

### Convert Labels to Integer

In [ ]:
labels = df["sentiment"].map({"positive": 1, "negative": 0})
labels.value_counts()

## 4. Preparing DataLoaders
### Train Test Splitting
80% train, 10% validation and 10% testing

In [ ]:
RANDOM_SEED = 42

seq_train, seq_sub, y_train, y_sub = train_test_split(
    all_review_seq, labels, test_size=0.2, random_state=RANDOM_SEED, stratify=labels
)

seq_val, seq_test, y_val, y_test = train_test_split(
    seq_sub, y_sub, test_size=0.5, random_state=RANDOM_SEED, stratify=y_sub
)

### Creating IMDB Datasets in Tensor

In [ ]:
class IMDBDataset(Dataset):
    """
    PyTorch Dataset class for IMDB movie reviews.

    Args:
        sequences: List of tokenized and padded text sequences.
        labels: Corresponding list of labels (floats).
    """

    def __init__(self, sequences: List[List[int]], labels: List[float]) -> None:
        self.sequences = torch.tensor(sequences, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.float)

    def __len__(self) -> int:
        return len(self.sequences)

     def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        return self.sequences[index], self.labels[index]

PyTorch's `torch.tensor()` constructor works with lists, tuples, or NumPy arrays, not pandas Series.

In [ ]:
BATCH_SIZE = 64
train_ds = IMDBDataset(seq_train, y_train.values)
val_ds = IMDBDataset(seq_val, y_val.values)
test_ds = IMDBDataset(seq_test, y_test.values)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

## 5. Neural Network Model Architectures
### Long Short-Term Memory (LSTM)
An LSTM architecture for sentiment analysis typically includes following structures:
- **Embedding Layer**:
    - Converts input token indices to dense vector embeddings.
    - Inputs shape: `(batch_size, sequence_length)`
    - Outputs shape: `(batch_size, sequence_length, embedding_size)`
- **LSTM Layer**:
    - Processes the embedded sequence to capture temporal depenencies.
    - Can be either single or multi-layer LSTM.
- **Classification Head**:
    - Usually a fully-connected (linear) layer projecting the hidden state(s) from LSTM to the output classes.
    - Often preceded by dropout for regularisation.
- **Activation and Loss**:
    - For binary sentiment, output logits go through sigmoid activation function with binary cross entropy loss.
    - For multi-class, softmax activation with cross entropy loss.

In [ ]:
class LSTMClassifier(nn.Module):
    """
    LSTM-based classifier for sequence classification tasks.

    Args:
        vocab_size: Size of the vocabulary.
        embedding_size: Dimension of word embeddings.
        hidden_size: Number of features in the hidden state of the LSTM.
        output_size: Number of output classes.
        n_layers: Number of LSTM layers. Defaults to 1.
        is_bidirectional: Whether to use bidirectional LSTM. Defaults to False.
        dropout_rate: Dropout rate for regularization. Defaults to 0.5.
    """

    def __init__(
        self,
        vocab_size: int,
        embedding_size: int,
        hidden_size: int,
        output_size: int,
        n_layers: int = 1,
        is_bidirectional: bool = False,
        dropout_rate: float = 0.5,
    ) -> None:
        super().__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.is_bidirectional = is_bidirectional

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_size
        )
        self.lstm = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            num_layers=n_layers,
            bidirectional=is_bidirectional,
            dropout=dropout_rate if n_layers > 1 else 0,
            batch_first=True,
        )
        self.dropout = nn.Dropout(dropout_rate)
        directional_factor = 2 if is_bidirectional else 1
        self.fc = nn.Linear(
            in_features=hidden_size * directional_factor,
            out_features=output_size,
        )

    def forward(
        self, text: torch.Tensor
    ) -> torch.Tensor:  # text shape: (batch_size, seq_len)
        """
        Forward propagation of the model.

        Args:
            text: Input tensor of shape (batch_size, seq_len) containing token indices.

        Returns:
            Output logits of shape (batch_size, output_size).
        """
        embedded = self.embedding(text)  # (batch_size, seq_len, embedding_size)
        lstm_out, (hidden, cell) = self.lstm(embedded)

        # If bidirectional, concatenate last forward and backward hidden states
        if self.lstm.bidirectional:
            hidden = torch.cat(
                (hidden[-2, :, :], hidden[-1, :, :]), dim=1
            )  # (batch_size, hidden_dim*2)
        else:
            hidden = hidden[-1, :, :]  # (batch_size, hidden_dim)

        dropped = self.dropout(hidden)
        out = self.fc(dropped)
        return out

### Transformer
A Transformer is a neural network architecture designed primarily for handling sequential data. Unlike traditional recurrent networks (RNNs/LSTMs), transformers use a self-attention mechanism to process the entire sequence at once. This allows the model to capture long-range dependencies efficiently and in parallel, which has made transformers the foundation for many state-of-art models in NLP, such as BERT and GPT.

**Features**:
- Instead of processing tokens sequentially, a transformer applies self-attention mechanism to let each token attend to every other token in the input sequence simultaneously.
- This is combined with positional encoding to inject information about token order, since the model doesn't inherently process inputs sequentially.
- The original Transformer architecture consists of an encoder and a decoder; for classification tasks like sentiment analysis, usually only the encoder part is used.
- Key components include multi-head attention layers, feed-forward neural networks, layer normalisation, and residual connections.

In [ ]:
class TransformerClassifier(nn.Module):
    """
    Transformer encoder-based classifier for sequence classification.

    Args:
        vocab_size: Size of the vocabulary.
        embed_dim: Dimension of the embedding vectors.
        nhead: Number of attention heads.
        num_layers: Number of transformer encoder layers.
        hidden_dim: Dimension of intermediate feed-forward layers.
        max_len: Maximum input sequence length.
        num_classes: Number of output classes. Defaults to 1.
        dropout_rate: Dropout rate. Defaults to 0.2.
    """

    def __init__(
        self,
        vocab_size: int,
        embed_dim: int,
        nhead: int,
        num_layers: int,
        hidden_dim: int,
        max_len: int,
        num_classes: int = 1,
        dropout_rate: float = 0.2,
    ) -> None:
        super().__init__()
        # Embedding Layer: Maps input tokens (0 to vocab_size-1) to dense vectors
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # Positional Encoding: Adds sine and cosine functions of different frequencies to embeddings (for token position in the sequence)
        self.pos_encoding = nn.Parameter(
            self._get_positional_encoding(max_len, embed_dim), requires_grad=False
        )
        # Transformer Encoder Layer: One layer of the transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,  # Number of attention heads
            dim_feedforward=hidden_dim,
            dropout=dropout_rate,
            batch_first=True,
        )
        # Stacks multiple encoder layers to form the full transformer encoder
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, num_layers=num_layers
        )

        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(embed_dim, num_classes)

    def _get_positional_encoding(self, seq_len: int, d_model: int) -> torch.Tensor:
        """
        Create fixed sinusoidal positional encodings.

        Args:
            seq_len: Sequence length.
            d_model: Embedding dimension.

        Returns:
            Positional encoding tensor of shape (1, seq_len, d_model).
        """
        pos = torch.arange(seq_len, dtype=torch.float32).unsqueeze(1)
        i = torch.arange(d_model, dtype=torch.float32).unsqueeze(0)
        angle_rates = 1 / torch.pow(10000, (2 * (i // 2)) / d_model)
        angle_rads = pos * angle_rates
        encoding = torch.zeros(seq_len, d_model)
        encoding[:, 0::2] = torch.sin(angle_rads[:, 0::2])
        encoding[:, 1::2] = torch.cos(angle_rads[:, 1::2])
        return encoding.unsqueeze(0)  # (1, seq_len, d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass for the transformer classifier.

        Args:
            x: Input tensor of shape (batch_size, seq_len).

        Returns:
            Output logits of shape (batch_size, num_classes).
        """
        # x: (batch_size, seq_len)
        embedded = self.embedding(x) + self.pos_encoding[:, : x.size(1), :]
        mask = x == 0  # Assuming 0 is the <PAD> index
        out = self.transformer_encoder(embedded, src_key_padding_mask=mask)
        pooled = out.mean(dim=1)  # mean pooling over sequence
        return self.fc(self.dropout(pooled))


In [ ]:
# Hyperparameters
VOCAB_SIZE = len(word2index)
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 128
OUTPUT_SIZE = 1  # For binary sentiment analysis (1 or 0)
N_LAYERS = 2
IS_BIDIRECTIONAL = True
DROPOUT_RATE = 0.5
N_EPOCHS = 5
LEARNING_RATE = 1e-3
N_HEAD = 4

In [ ]:
lstm = LSTMClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    output_size=OUTPUT_SIZE,
    n_layers=N_LAYERS,
    is_bidirectional=IS_BIDIRECTIONAL,
    dropout_rate=DROPOUT_RATE,
)

transformer = TransformerClassifier(
    vocab_size=VOCAB_SIZE,
    embed_dim=EMBEDDING_SIZE,
    nhead=N_HEAD,
    num_layers=N_LAYERS,
    hidden_dim=HIDDEN_SIZE,
    max_len=SEQUENCE_LENGTH,
    dropout_rate=DROPOUT_RATE,
)

## Evaluation Metrics
### Accuracy
Accuracy is the most common evaluation metric for classification problems, representing the percentage of correct predictions out of total predictions. It provides a simple measure of how often the classifier makes correct predictions across all classes.

\begin{align*}
\text{Accuracy} = \dfrac{\text{True Positives (TP)} + \text{True Negatives (TN)}}{\text{Total Samples}}
\end{align*}

In [ ]:
accuracy = Accuracy(
    task="binary",
    num_classes=2,
).to(device)

## Loss Function
### Binary Cross-Entropy
Binary Cross-Entropy loss function is used to measure the error between the true labels $y$ and the predicted probabilities $\hat y$. The loss function is defined as:

\begin{align*}
L = - \dfrac{1}{n} \sum_{i=1}^{n} y_{i} \log(\hat y_{i}) + (1-y_{i}) \log(1- \hat y_{i})
\end{align*}

In [ ]:
criterion = nn.BCEWithLogitsLoss()

## Optimiser
An optimiser in neural networks is used to adjust the parameters (weights and biases) of a model during training to minimise the loss. Optimisers are essential for enabling neural networks to learn from data: without them, the model would not improve over time.

In [ ]:
lstm_optim = optim.Adam(lstm.parameters(), lr=LEARNING_RATE)
transformer_optim = optim.Adam(
    transformer.parameters(), lr=LEARNING_RATE, weight_decay=1e-5
)

## Training and Evaluation
1. Iterate through epochs
1. For each epoch, iterate through training batches, perform training steps, calculate the train loss and evaluation metrics per batch.
1. For each epoch, iterate through testing batches, perform testing steps, calculate the test loss and evaluation metrics per batch.
1. Store the results.

### Training Steps
1. Zero the gradients
    - Clear the gradients from the previous iteration to prevent accumulation across batches.
1. Forward pass
    - Pass inputs through the model to obtain predictions.
1. Calculate loss and evaluation metrics per batch
    - Measure how far the predictions deviate from the true labels using a loss function.
    - Compute evaluation metrics (e.g., accuracy, F1 Score, etc.) for the current batch.
1. Backward pass
    - Compute gradients of the loss with respect to the model's parameters via backpropagation.
    - Update the parameter $\theta$ using the computed gradients, typically following:
    $$
        \theta \leftarrow \theta - \eta \dfrac{\partial \mathcal{L}}{\partial \theta}
    $$
    where $\eta$ is the learning rate.
1. Average training loss and evaluation metrics
    - Calculate the mean loss and metric values across all batches in the epoch.

In [ ]:
def train_step(
    model: nn.Module,
    data_loader: DataLoader,
    criterion: nn.Module,
    optimiser: optim.Optimizer,
    accuracy: Accuracy,
    device: torch.device,
) -> tuple[float, float]:
    """
    Perform one training epoch step.

    Args:
        model: The model to train.
        data_loader: DataLoader for training data.
        criterion: Loss function.
        optimiser: Optimizer.
        accuracy: Torchmetrics accuracy metric.
        device: Device to run the computations on.

    Returns:
        Average training loss and training accuracy (percentage).
    """
    model.train()
    accuracy.reset()
    train_loss = 0.0
    total_samples = 0

    for texts, labels in data_loader:
        texts, labels = texts.to(device), labels.to(device).unsqueeze(1)

        # Optimiser zero grad without intervening forward pass
        optimiser.zero_grad()

        # Forward pass
        y_logits = model(texts)  # Shape (batch_size, 1)

        # Calculate loss
        loss = criterion(y_logits, labels)
        train_loss += loss.item() * texts.size(0)
        total_samples += texts.size(0)

        # Calculate accuracy
        y_probs = torch.sigmoid(y_logits)
        y_preds = (y_probs >= 0.5).int()
        accuracy.update(y_preds, labels.int())

        # Loss backward for backpropagation (computing gradients)
        loss.backward()

        # Optimiser step to apply gradients and update parameters
        optimiser.step()
    avg_train_loss = train_loss / total_samples
    train_acc = accuracy.compute().item() * 100
    return avg_train_loss, train_acc

### Testing Steps
1. Forward pass
    - Set the model to evaluation mode (which disables dropout and batch normalisation and deactivates gradient tracking for safety).
    - Pass inputs through the model to obtain predictions.

1. Calculate loss and evaluation metrics per batch
    - Measure how far the predictions deviate from the true labels using the loss function.
    - Compute evaluation metrics (e.g., accuracy, F1-Score, etc.) for the current batch.

1. Average test loss and evaluation metrics
    - Calculate the mean loss and metric values across all batches in the epoch.

In [ ]:
def validation_step(
    model: nn.Module,
    data_loader: DataLoader,
    criterion: nn.Module,
    accuracy: Accuracy,
    device: torch.device,
) -> tuple[float, float]:
    """
    Perform one validation epoch step.

    Args:
        model: The model to validate.
        data_loader: DataLoader for validation data.
        criterion: Loss function.
        accuracy: Torchmetrics accuracy metric.
        device: Device to run the computations on.

    Returns:
        Average validation loss and validation accuracy (percentage).
    """
    model.eval()
    accuracy.reset()
    val_loss = 0.0
    total_samples = 0
    with torch.inference_mode():
        for texts, labels in data_loader:
            texts, labels = texts.to(device), labels.to(device).unsqueeze(-1)

            # 1. Forward Pass
            y_logits = model(texts)

            # 2. Calculate loss
            loss = criterion(y_logits, labels)
            val_loss += loss.item() * texts.size(0)
            total_samples += texts.size(0)

            # 3. Calculate accuracy
            y_probs = torch.sigmoid(y_logits)
            y_preds = (y_probs >= 0.5).int()
            accuracy.update(y_preds, labels.int())

    avg_val_loss = val_loss / total_samples
    val_acc = accuracy.compute().item() * 100
    return avg_val_loss, val_acc


### Model Training and Evaluation Pipeline

In [ ]:
def train_and_validate(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    criterion: nn.Module,
    optimiser: optim.Optimizer,
    device: torch.device,
    total_epochs: int,
) -> tuple[Dict[str, List[float]], Dict[str, List[float]]]:
    """
    Train and validate the model over a specified number of epochs.

    Args:
        model: The model to train.
        train_loader: DataLoader for training data.
        val_loader: DataLoader for validation data.
        criterion: Loss function.
        optimiser: Optimizer.
        device: Device to run computations on.
        total_epochs: Number of training epochs.

    Returns:
        Dictionaries containing lists of training and validation losses and accuracies for each epoch.
    """
    model.to(device)
    epochs_range = range(1, total_epochs + 1)
    train_results = {"Loss": [], "Accuracy": []}
    val_results = {"Loss": [], "Accuracy": []}

    start_time = time.time()

    for epoch in epochs_range:
        train_loss, train_acc = train_step(
            data_loader=train_loader,
            model=model,
            criterion=criterion,
            optimiser=optimiser,
            accuracy=accuracy,
            device=device,
        )
        train_results["Loss"].append(train_loss)
        train_results["Accuracy"].append(train_acc)

        val_loss, val_acc = validation_step(
            data_loader=val_loader,
            model=model,
            criterion=criterion,
            accuracy=accuracy,
            device=device,
        )
        val_results["Loss"].append(val_loss)
        val_results["Accuracy"].append(val_acc)

        print(
            f"Epoch: {epoch}/{total_epochs} - "
            f"Train Loss: {train_loss:.4f}  Train Accuracy: {train_acc:.2f}% | "
            f"Val Loss: {val_loss:.4f}  Val Accuracy: {val_acc:.2f}%"
        )
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training and validation completed in {elapsed_time:.2f} seconds.")
    return train_results, val_results

### Performance Evaluation (LSTM)

In [ ]:
print("Training LSTM...")
lstm_train_results, lstm_val_results = train_and_validate(
    lstm,
    train_loader,
    val_loader,
    criterion,
    lstm_optim,
    device,
    N_EPOCHS,
)

### Performance Evaluation (Transformer)

In [ ]:
print("Training Transformer...")
transformer_train_results, transformer_val_results = train_and_validate(
    transformer,
    train_loader,
    val_loader,
    criterion,
    transformer_optim,
    device,
    N_EPOCHS,
)

## Results
### Overall Performance

In [ ]:
def plot_performance(
    model_name: str,
    total_epochs: int,
    train_metrics: Dict[str, List[float]],
    test_metrics: Dict[str, List[float]],
) -> None:
    """
    Plot training and testing metrics over epochs.

    Args:
        model_name: Name of the model for plot titles.
        total_epochs: Number of training epochs.
        train_metrics: Dictionary of training metrics.
        test_metrics: Dictionary of testing metrics.
    """
    epochs_range = range(1, total_epochs + 1)
    metric_names = list(train_metrics.keys())
    n_metrics = len(metric_names)
    print(metric_names)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    axes = axes.flatten()

    for i, metric_name in enumerate(metric_names):
        ax = axes[i]
        ax.plot(
            epochs_range, train_metrics[metric_name], label=f"Train {metric_name}"
        )  # Train metric
        ax.plot(
            epochs_range, test_metrics[metric_name], label=f"Test {metric_name}"
        )  # Test metric
        ax.set_title(f"{metric_name} Over Epochs ({model_name})", fontsize=15)
        ax.legend()
        ax.set_xlabel("Epoch")
        ax.set_ylabel(metric_name)

    if n_metrics < len(axes):
        for j in range(n_metrics, len(axes)):
            plt.delaxes(axes[j])
    plt.tight_layout()
    plt.show()

In [ ]:
plot_performance("LSTM", N_EPOCHS, lstm_train_results, lstm_val_results)
plot_performance(
    "Transformer", N_EPOCHS, transformer_train_results, transformer_val_results
)

### Predictions on Test DataLoader

In [ ]:
def make_predictions(
    model: nn.Module,
    data_loader: DataLoader,
    criterion: nn.Module,
    accuracy: Accuracy,
    device: torch.device,
):
    all_predictions = []
    all_labels = []
    test_loss = 0.0
    total_samples = 0
    accuracy.reset()
    model.eval()
    with torch.inference_mode():
        for texts, labels in data_loader:
            texts, labels = texts.to(device), labels.to(device).unsqueeze(-1)

            # Forward pass
            y_logits = model(texts)
            y_probs = torch.sigmoid(y_logits)
            y_preds = (y_probs >= 0.5).int()

            # Calculate accuracy
            accuracy.update(y_preds, labels.int())

            # Update lists
            all_predictions.append(y_preds.cpu())
            all_labels.append(labels.cpu())

            # Calculate loss
            loss = criterion(y_logits.view_as(labels), labels)
            test_loss += loss.item() * texts.size(0)
            total_samples += texts.size(0)
    avg_test_loss = test_loss / total_samples
    test_acc = accuracy.compute().item() * 100
    print(f"Test loss: {avg_test_loss:.4f} | Test acc: {test_acc:.2f}%")
    all_preds_tensor = torch.cat(all_predictions)
    all_labels_tensor = torch.cat(all_labels)
    return all_preds_tensor, all_labels_tensor


In [ ]:
print("Testing LSTM...")
all_preds_lstm, all_labels_lstm = make_predictions(
    model=lstm,
    data_loader=test_loader,
    criterion=criterion,
    accuracy=accuracy,
    device=device,
)

wrong_indices_lstm = (all_preds_lstm != all_labels_lstm).nonzero(as_tuple=True)[0]
print(
    f"Number of failed predictions: {len(wrong_indices_lstm)}/{len(all_labels_lstm)} ({100.0 * len(wrong_indices_lstm) / len(all_labels_lstm):.2f}%)"
)

In [ ]:
print("Testing Transformer...")
all_preds_transformer, all_labels_transformer = make_predictions(
    model=transformer,
    data_loader=test_loader,
    criterion=criterion,
    accuracy=accuracy,
    device=device,
)

wrong_indices_transformer = (all_preds_transformer != all_labels_transformer).nonzero(
    as_tuple=True
)[0]
print(
    f"Number of failed predictions: {len(wrong_indices_transformer)}/{len(all_labels_transformer)} ({100.0 * len(wrong_indices_transformer) / len(all_labels_transformer):.2f}%)"
)

### Confusion Matrix

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# Convert to tensors if not already
true_labels = all_labels_lstm
preds_tensor_lstm = all_preds_lstm.detach().clone()
preds_tensor_transformer = all_preds_transformer.detach().clone()
class_names = ["Negative", "Positive"]
model_names = ["LSTM", "Transformer"]

# Compute confusion matrix
conf_matrix = ConfusionMatrix(num_classes=2, task="binary")
conf_matrix_tensor_lstm = conf_matrix(preds_tensor_lstm, true_labels)
conf_matrix_tensor_transformer = conf_matrix(preds_tensor_transformer, true_labels)

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(16, 10))
axes = axes.flatten()

plot_confusion_matrix(
    conf_mat=conf_matrix_tensor_lstm.numpy(),
    cmap="Blues",
    axis=axes[0],
)
plot_confusion_matrix(
    conf_mat=conf_matrix_tensor_transformer.numpy(),
    cmap="Blues",
    axis=axes[1],
)

for i, ax in enumerate(axes):
    ax.set_xticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_yticks(range(len(class_names)))
    ax.set_yticklabels(class_names)
    ax.set_xlabel("Predicted label")
    ax.set_ylabel("True label")
    ax.set_title(f"Confusion Matrix ({model_names[i]})", fontdict={"fontsize": 16})

plt.tight_layout()
plt.show()

## Sample Predictions

In [ ]:
def predict_single_text(
    model: torch.nn.Module,
    raw_text: str,
    word2index: Dict[str, int],
    max_seq_len: int,
    device: torch.device,
) -> Tuple[int, float]:
    """
    Predict the label and confidence score for a single raw text input using the given model.

    Args:
        model: The trained PyTorch model for classification.
        raw_text: The raw input string to classify.
        word2index: Mapping from token strings to their integer indices.
        max_seq_len: The fixed input sequence length for the model.
        device: The device on which to perform computation (CPU or GPU).

    Returns:
        Tuple:
            pred: Predicted class label (0 or 1).
            confidence: Confidence score between 0 and 1 indicating certainty about prediction.
    """
    model.eval()
    cleaned = clean_text(raw_text)
    tokens = cleaned.split()
    indices = [word2index.get(word, word2index["<UNK>"]) for word in tokens]
    if len(indices) < max_seq_len:
        indices += [word2index["<PAD>"]] * (max_seq_len - len(indices))
    else:
        indices = indices[:max_seq_len]
    input_tensor = torch.tensor([indices], dtype=torch.long).to(device)
    with torch.no_grad():
        logits = model(input_tensor)
        prob = torch.sigmoid(logits).item()
        pred = int(prob >= 0.5)
        confidence = abs(prob - 0.5) * 2
    return pred, confidence

In [ ]:
text_1 = "I really enjoyed this movie! The storyline was engaging and the characters were well-developed. The visuals were stunning, and the soundtrack perfectly matched the mood. Definitely worth watching!"

pred_lstm, confidence_lstm = predict_single_text(
    lstm, text_1, word2index, SEQUENCE_LENGTH, device
)
pred_transformer, confidence_transformer = predict_single_text(
    transformer, text_1, word2index, SEQUENCE_LENGTH, device
)
class_map = {0: "Negative", 1: "Positive"}

print(f"Text: {text_1}\n")
print(
    f"LSTM:\t\t Predicted class: {pred_lstm}({class_map[pred_lstm]}), Confidence: {confidence_lstm:.3f}"
)
print(
    f"Transformer:\t Predicted class: {pred_transformer}({class_map[pred_transformer]}), Confidence: {confidence_transformer:.3f}"
)

In [ ]:
text_2 = "I was quite disappointed by this movie. The story felt predictable and dull, the characters were underdeveloped, and the pacing was painfully slow. I struggled to stay interested and wouldn't recommend it."

pred_lstm, confidence_lstm = predict_single_text(
    lstm, text_2, word2index, SEQUENCE_LENGTH, device
)
pred_transformer, confidence_transformer = predict_single_text(
    transformer, text_2, word2index, SEQUENCE_LENGTH, device
)
class_map = {0: "Negative", 1: "Positive"}

print(f"Text: {text_2}\n")
print(
    f"LSTM:\t\t Predicted class: {pred_lstm}({class_map[pred_lstm]}), Confidence: {confidence_lstm:.3f}"
)
print(
    f"Transformer:\t Predicted class: {pred_transformer}({class_map[pred_transformer]}), Confidence: {confidence_transformer:.3f}"
)

## Pre-Trained Transformer Model (From Hugging Face)

To use pre-trained transformer models from Hugging Face, the data preparation pipelines (from loading data to fine-tuning models) should be done using `datasets`.

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    path="csv",
    data_files={"all_data": "_datasets/IMDB_Dataset.csv"},
)

dataset = dataset.rename_column("sentiment", "labels")

In [ ]:
def label_to_int(sample: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert string sentiment labels ('positive'/'negative') to integer labels (1/0).

    Args:
        sample (Dict[str, Any]): A dictionary representing a data example with a 'labels' key.

    Returns:
        Dict[str, Any]: The same dictionary with 'labels' converted to integer values.
    """

    if sample["labels"] == "positive":
        sample["labels"] = 1
    else:
        sample["labels"] = 0
    return sample


# Apply it on the full dataset
dataset = dataset.map(label_to_int)

In [ ]:
RANDOM_SEED = 42

# 80% train, 10% validation and 10% test
split = dataset["all_data"].train_test_split(test_size=0.2, seed=RANDOM_SEED)
train_dataset = split["train"]
subset = split["test"]

subset_split = subset.train_test_split(test_size=0.5, seed=RANDOM_SEED)
val_dataset = subset_split["train"]
test_dataset = subset_split["test"]

### Tokenisation with Hugging Face

In [ ]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokeniser = AutoTokenizer.from_pretrained(model_name)


def tokenise(batch: Dict[str, Any]) -> Dict[str, Any]:
    """
    Tokenise a batch of text data using a tokeniser.

    Args:
        batch: A batch dictionary with key 'review' containing text string(s).

    Returns:
        Tokenised output including token ids, attention mask, etc.
    """
    return tokeniser(
        batch["review"],
        padding="max_length",
        truncation=True,
    )

In [ ]:
tokenised_train_dataset = train_dataset.map(tokenise, batched=True)
tokenised_val_dataset = val_dataset.map(tokenise, batched=True)
tokenised_test_dataset = test_dataset.map(tokenise, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokeniser)

### Reformatting for PyTorch

In [ ]:
tokenised_train_dataset.set_format(
    "torch", columns=["input_ids", "attention_mask", "labels"]
)
tokenised_val_dataset.set_format(
    "torch", columns=["input_ids", "attention_mask", "labels"]
)
tokenised_test_dataset.set_format(
    "torch", columns=["input_ids", "attention_mask", "labels"]
)

### Training Arguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Evaluate after each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
)

### Metrics

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(
    eval_pred: Tuple[NDArray[np.float64], NDArray[np.int64]],
) -> Dict[str, float]:
    """
    Compute evaluation metrics for model outputs and true labels.

    Args:
        eval_pred: Tuple containing raw model logits and target labels.
            - logits: Array of shape (batch_size, num_classes)
            - labels: Array of shape (batch_size,)

    Returns:
        Dictionary of computed metric scores, e.g., {'accuracy': 0.95}.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### Loading Pre-Trained Model

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2  # Binary Classification
hf_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)

### Training Transformer Model

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=hf_model,
    args=training_args,
    train_dataset=tokenised_train_dataset,
    eval_dataset=tokenised_val_dataset,
    processing_class=tokeniser,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.save_model("./my_finetuned_model")
tokeniser.save_pretrained("./my_finetuned_model")

In [ ]:
# Evaluate on test set
metrics = trainer.evaluate(test_dataset)
print(metrics)

In [ ]:
MY_MODEL_PATH = "./my_finetuned_model"
my_tokeniser = AutoTokenizer.from_pretrained(MY_MODEL_PATH)
my_model = AutoModelForSequenceClassification.from_pretrained(MY_MODEL_PATH)

inputs = my_tokeniser(text_1, return_tensors="pt")
with torch.no_grad():
    logits = my_model(**inputs).logits
predicted_class_id = logits.argmax().item()

print(
    f"Predicted: {predicted_class_id} ({my_model.config.id2label[predicted_class_id]})"
)

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=MY_MODEL_PATH)
print(classifier(text_1))